In [1]:
import sys
import os
os.environ["MASTER_ADDR"] = "SH-IDC1-10-140-37-139"
os.environ["MASTER_PORT"] = "33212"
sys.argv = [
    'debug.py', 
    '/mnt/petrelfs/liyizhuo/projects/vindlu/exp/exp_pretrain_blip/blip_T5_5m_16x128/config.py', 
    'wandb.enable', 
    'False', 
    'output_dir', 
    'tmp',
    'evaluate',
    'True',
    'model.t5_model_name', 
    'google/flan-t5-xl',
    'pretrained_path',
    'exp/exp_pretrain_blip/blip_T5_5m_16x128/blip_T5_xl_5m_16x128/ckpt_best.pth',
    ]

import torch
import torch.nn.functional as F

from transformers import BertTokenizer

from models.vindlu_blip_T5 import VindLU_BLIP_T5
from utils.config_utils import setup_main

config = setup_main()

1


remaining dirs or files: ['config.json', 'train.log']


2023-04-04T17:21:44 | vindlu: Logging to: tmp/train.log
2023-04-04T17:21:44 | utils.config_utils: config: {
  data_dir: /mnt/petrelfs/share/videointern/annotations/
  data_root: /mnt/petrelfs/share/videointern/annotations/videos_images
  anno_root_pt: /mnt/petrelfs/share/videointern/annotations/anno_pretrain
  anno_root_downstream: /mnt/petrelfs/share/videointern/annotations/anno_downstream
  available_corpus: {
      cc3m: ['/mnt/petrelfs/share/videointern/annotations/anno_pretrain/cc3m_train.json', 'pssd:s3://GCC']
      cc12m: ['/mnt/petrelfs/share/videointern/annotations/anno_pretrain/cc12m_train.json', 'pssd:s3://GCC/GCC12m']
      sbu: ['/mnt/petrelfs/share/videointern/annotations/anno_pretrain/sbu.json', 'pssd:s3://SBU/']
      vg: ['/mnt/petrelfs/share/videointern/annotations/anno_pretrain/vg.json', 'pssd:s3://VG_dataset/']
      coco: ['/mnt/petrelfs/share/videointern/annotations/anno_pretrain/coco.json', 'pssd:s3://coco_caption']
      imagenet1k: ['/mnt/petrelfs/share/videoi

In [2]:
from tasks.shared_utils import setup_model
from tasks.pretrain import setup_dataloaders
from utils.basic_utils import setup_seed

is_pretrain = config.mode == "pt"

setup_seed(42)
device = torch.device(config.device)

train_loaders, test_name2loaders, train_media_types = setup_dataloaders(
    config, mode=config.mode
)
num_steps_per_epoch = sum(len(d) for d in train_loaders)
config.scheduler.num_training_steps = num_steps_per_epoch * config.scheduler.epochs
config.scheduler.num_warmup_steps = num_steps_per_epoch * config.scheduler.warmup_epochs

model_cls = eval(config.model.get('model_cls', 'VindLU'))
(
    model,
    model_without_ddp,
    optimizer,
    scheduler,
    scaler,
    tokenizer,
    start_epoch,
    global_step,
) = setup_model(
config,
model_cls=model_cls,
has_decoder=False,
pretrain=is_pretrain,
find_unused_parameters=False if "VindLU_BLIP" in \
    config.model.get('model_cls', 'VindLU') else True,
)

2023-04-04T17:21:50 | tasks.pretrain: Creating dataset for pt
2023-04-04T17:21:50 | dataset.sqlite_dataset: Load json file
2023-04-04T17:22:11 | dataset.sqlite_dataset: Num samples: 2881167
2023-04-04T17:22:11 | dataset.sqlite_dataset: Num samples too short: 0
2023-04-04T17:22:11 | dataset.sqlite_dataset: Load json file
2023-04-04T17:22:34 | dataset.sqlite_dataset: Num samples: 2487516
2023-04-04T17:22:34 | dataset.sqlite_dataset: Num samples too short: 9634


Loading /mnt/petrelfs/share/videointern/annotations/anno_downstream/msrvtt_qa_val.json: 100%|██████████| 12278/12278 [00:00<00:00, 425963.13it/s]

2023-04-04T17:22:35 | tasks.shared_utils: Creating model


2023-04-04T17:23:00 | models.vindlu_blip_qformer: Build vision_encoder: clip_vision_b16
2023-04-04T17:23:00 | models.backbones.vit.clip_vision: Return Layer: 1
2023-04-04T17:23:00 | models.backbones.vit.clip_vision: Return Interval: 1
2023-04-04T17:23:00 | models.backbones.vit.clip_vision: With LayerNorm and Proj: False
2023-04-04T17:23:01 | models.backbones.vit.clip_vision: Teacher return index: [11]
2023-04-04T17:23:01 | models.backbones.vit.clip_vision: Load pretrained weights from /mnt/petrelfs/share_data/likunchang/model/clip_visual_encoder/vit_b16.pth
2023-04-04T17:23:03 | models.backbones.vit.clip_vision: Inflate: conv1.weight, torch.Size([768, 3, 16, 16]) => torch.Size([768, 3, 1, 16, 16])
2023-04-04T17:23:03 | models.backbones.vit.clip_vision: Init center: True
2023-04-04T17:23:03 | models.backbones.vit.clip_vision: Unexpected keys: ['proj', 'ln_post.weight', 'ln_post.bias']
2023-04-04T17:23:03 | models.backbones.vit.clip_vision: Missing keys: []
2023-04-04T17:23:23 | models.v

2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.2.attention.self.value.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.2.attention.self.value.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.2.attention.output.dense.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.2.attention.output.dense.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.2.attention.output.LayerNorm.weight: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.2.attention.output.LayerNorm.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.2.crossattention.self.query.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.2

2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.4.intermediate_query.dense.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.4.output_query.dense.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.4.output_query.dense.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.4.output_query.LayerNorm.weight: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.4.output_query.LayerNorm.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.5.attention.self.query.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.5.attention.self.query.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.5.attention.self.key.we

2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.7.output_query.dense.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.7.output_query.dense.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.7.output_query.LayerNorm.weight: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.7.output_query.LayerNorm.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.8.attention.self.query.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.8.attention.self.query.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.8.attention.self.key.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.8.attention.self.key.bia

2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.10.crossattention.self.key.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.10.crossattention.self.value.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.10.crossattention.self.value.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.10.crossattention.output.dense.weight: wd: 0.02, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.10.crossattention.output.dense.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.10.crossattention.output.LayerNorm.weight: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param module.qformer.bert.encoder.layer.10.crossattention.output.LayerNorm.bias: wd: 0, lr: 0.0001
2023-04-04T17:25:31 | utils.optimizer: param modul

2023-04-04T17:25:41 | tasks.shared_utils: Loaded checkpoint from exp/exp_pretrain_blip/blip_T5_5m_16x128/blip_T5_xl_5m_16x128/ckpt_best.pth


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = VindLU_BLIP_T5(config, tokenizer=tokenizer)

# model.cuda()

In [3]:
data = next(iter(test_name2loaders['msrvtt_qa_val'].dataset))

In [4]:
image, text, idx = data

print(image.shape)
print(text)

image = image.unsqueeze(0).cuda()
text = [text]

torch.Size([4, 3, 224, 224])
what are three people sitting on


In [5]:
print("Full precision")
model(image, None, idx=None, raw_text=text, log_generation=True)
print("Float16")
with torch.cuda.amp.autocast(enabled=config.fp16, dtype=torch.float16):
    model(image, None, idx=None, raw_text=text, log_generation=True)
print("BFloat16")
with torch.cuda.amp.autocast(enabled=config.fp16, dtype=torch.bfloat16):
    model(image, None, idx=None, raw_text=text, log_generation=True)

Full precision
2023-04-04T17:25:47 | models.vindlu_blip_T5: raw_text_input: what are three raw_text_output: people sitting on generated_text: [0, 3567, 692, 16, 8] 
Float16
2023-04-04T17:25:47 | models.vindlu_blip_T5: raw_text_input: what raw_text_output: are three people sitting on generated_text: [0, 0, 0, 0, 0] 
BFloat16
2023-04-04T17:25:48 | models.vindlu_blip_T5: raw_text_input: what raw_text_output: are three people sitting on generated_text: [0, 3, 9, 248, 706] 


In [ ]:
# Data
image = torch.zeros(1, 4, 3, 224, 224)
image = image.cuda()
max_input_len, max_output_len = 25, 5
raw_question = raw_text_input = ["What is in the picture?"]
raw_answer = raw_text_output = ["A dog"]

In [ ]:
self = model.module

_, input_t5 = self.encode_vision(image)
input_t5 = self.t5_proj(input_t5)
atts_t5 = torch.ones(input_t5.size()[:-1], dtype=torch.long).to(input_t5.device)

input_tokens = self.tokenize(raw_text_input, input_t5.device, max_input_len)
output_tokens = self.tokenize(
    raw_text_output, input_t5.device, max_output_len
)

encoder_atts = torch.cat([atts_t5, input_tokens.attention_mask], dim=1)

targets = output_tokens.input_ids.masked_fill(
    output_tokens.input_ids == self.t5_tokenizer.pad_token_id, -100
)

inputs_embeds = self.t5_model.encoder.embed_tokens(input_tokens.input_ids)
inputs_embeds = torch.cat([input_t5, inputs_embeds], dim=1)

In [ ]:
generated_text = self.generate(inputs_embeds, encoder_atts)
print(generated_text)

In [ ]:
self.t5_tokenizer.batch_decode(generated_text)